In [1]:
import os
import re
import sklearn
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *
pd.set_option('display.max_columns', 500)

Load Data

Season Detailed Results

In [2]:
dstats = pd.read_csv('/Users/seancollins/Desktop/DATA 3000 Project/mens-march-mania-2022/MDataFiles_Stage2/MRegularSeasonDetailedResults.csv')

In [3]:
dstats.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [4]:
#home and away
dstats['WLoc'] = dstats['WLoc'].apply(lambda x: 1 if x=='H' else 0)
dstats['LLoc'] = 1 - dstats['WLoc']

In [5]:
dstats.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,LLoc
0,2003,10,1104,68,1328,62,0,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,1
1,2003,10,1272,70,1393,63,0,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16,1
2,2003,11,1266,73,1437,61,0,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23,1
3,2003,11,1296,56,1457,50,0,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23,1
4,2003,11,1400,77,1208,71,0,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14,1


In [6]:
dstats.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
       'LLoc'],
      dtype='object')

In [7]:
wcols = ['Season', 'WTeamID', 'WScore', 'WLoc',
         'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM',
           'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 
           'WStl', 'WBlk', 'WPF']
winners_df = dstats[wcols].rename(columns={
    'WTeamID' : 'TeamID', 'WScore' : 'Score', 
    'WLoc' : 'Loc', 'WFGM':'FGM', 'WFGA':'FGA',
    'WFGM3':'FGM3', 'WFGA3':'FGA3', 'WFTM':'FTM',
    'WFTA':'FTA', 'WOR':'OR', 'WDR':'DR', 
    'WAst':'Ast', 'WTO':'TO', 'WStl':'Stl', 
    'WBlk':'Blk', 'WPF':'PF'
})

In [8]:
lcols = [
    'Season', 'LTeamID', 'LScore', 'LLoc',
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM',
    'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 
    'LStl', 'LBlk', 'LPF'
]
losers_df = dstats[lcols].rename(columns={
    'LTeamID' : 'TeamID', 'LScore' : 'Score', 
    'LLoc' : 'Loc', 'LFGM':'FGM', 'LFGA':'FGA',
    'LFGM3':'FGM3', 'LFGA3':'FGA3', 'LFTM':'FTM',
    'LFTA':'FTA', 'LOR':'OR', 'LDR':'DR', 
    'LAst':'Ast', 'LTO':'TO', 'LStl':'Stl', 
    'LBlk':'Blk', 'LPF':'PF'
})

In [9]:
winners_df.head()

,Season,TeamID,Score,Loc,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
0,2003,1104,68,0,27,58,3,14,11,18,14,24,13,23,7,1,22
1,2003,1272,70,0,26,62,8,20,10,19,15,28,16,13,4,4,18
2,2003,1266,73,0,24,58,8,18,17,29,17,26,15,10,5,2,25
3,2003,1296,56,0,18,38,3,9,17,31,6,19,11,12,14,2,18
4,2003,1400,77,0,30,61,6,14,11,13,17,22,12,14,4,4,20


In [10]:
losers_df.head()

,Season,TeamID,Score,Loc,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
0,2003,1328,62,1,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,1393,63,1,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,1437,61,1,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,1457,50,1,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,1208,71,1,24,62,6,16,17,27,21,15,12,10,7,1,14


In [11]:
a = pd.concat([winners_df, losers_df]).groupby(['Season', 'TeamID']).agg(Num=('TeamID', 'count'))
b = pd.concat([winners_df, losers_df]).groupby(['Season', 'TeamID']).sum()
all_df = pd.concat([a,b], axis=1)
all_df.head()

Num  Score  Loc  FGM   FGA  FGM3  FGA3  FTM  FTA   OR   DR  \
Season TeamID                                                               
2003   1102     28   1603   15  536  1114   219   583  312  479  117  471   
       1103     27   2127   14  733  1508   147   434  514  698  264  538   
       1104     28   1940   16  673  1601   178   556  416  586  380  670   
       1105     26   1866   12  634  1602   197   540  401  568  351  601   
       1106     28   1781   14  656  1548   171   494  298  461  344  668   

               Ast   TO  Stl  Blk   PF  
Season TeamID                           
2003   1102    364  320  167   50  525  
       1103    411  341  196   63  536  
       1104    339  372  185  106  505  
       1105    378  485  242   54  526  
       1106    327  477  234   88  509

In [12]:
#calculate true shooting 
all_df['TS'] = round(all_df['Score']/(2*(all_df['FGA']+0.44*all_df['FTA'])),3)

#calculate turnover %%
all_df['TO%'] = round(all_df['TO']/(all_df['FGA']+0.44*all_df['FTA']+all_df['TO']),3)

#field goal percentage
all_df['FGpct'] = round(all_df['FGM']/all_df['FGA'],3)

#3 pt percentage
all_df['3FGpct'] = round(all_df['FGM3']/all_df['FGA3'],3)

#assist to turnover ratio
all_df['AST/TO'] = round(all_df['Ast']/all_df['TO'],3)

#Free throw percentage
all_df['FTpct'] = round(all_df['FTM']/all_df['FTA'],3)

#assist to steal ratio
all_df['AST/STL'] = round(all_df['Ast']/all_df['Stl'],3)

all_df = all_df.reset_index()

In [13]:
all_df.head()

,Season,TeamID,Num,Score,Loc,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,TS,TO%,FGpct,3FGpct,AST/TO,FTpct,AST/STL
0,2003,1102,28,1603,15,536,1114,219,583,312,479,117,471,364,320,167,50,525,0.605,0.195,0.481,0.376,1.138,0.651,2.180
1,2003,1103,27,2127,14,733,1508,147,434,514,698,264,538,411,341,196,63,536,0.586,0.158,0.486,0.339,1.205,0.736,2.097
2,2003,1104,28,1940,16,673,1601,178,556,416,586,380,670,339,372,185,106,505,0.522,0.167,0.420,0.320,0.911,0.710,1.832
3,2003,1105,26,1866,12,634,1602,197,540,401,568,351,601,378,485,242,54,526,0.504,0.208,0.396,0.365,0.779,0.706,1.562
4,2003,1106,28,1781,14,656,1548,171,494,298,461,344,668,327,477,234,88,509,0.509,0.214,0.424,0.346,0.686,0.646,1.397


In [14]:
team_3pct = all_df[['Season', 'TeamID', '3FGpct']]

In [17]:
dstats.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,LLoc
0,2003,10,1104,68,1328,62,0,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,1
1,2003,10,1272,70,1393,63,0,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16,1
2,2003,11,1266,73,1437,61,0,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23,1
3,2003,11,1296,56,1457,50,0,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23,1
4,2003,11,1400,77,1208,71,0,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14,1


In [19]:
#point differential
dstats['ScoreGap'] = dstats['WScore'] - dstats['LScore']

In [20]:
#some additional columns
win = dstats.groupby(['Season', 'WTeamID']).agg(WNum=('DayNum', 'count'),
                                                AvgWpts=('ScoreGap', 'mean'),
                                                TotalWPoints=('WScore', 'sum'),
                                                ).reset_index().rename(columns={'WTeamID' : 'TeamID'})
loss = dstats.groupby(['Season', 'LTeamID']).agg(LNum=('DayNum', 'count'),
                                                AvgLpts=('ScoreGap', 'mean'),
                                                TotalLPoints=('LScore', 'sum'),
                                                ).reset_index().rename(columns={'LTeamID' : 'TeamID'})


In [21]:
df_features_d = win.merge(loss, on=['Season','TeamID'], how='outer').drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop=True)
df_features_d.fillna(0, inplace=True)

In [22]:
df_features_d.head(

)

,Season,TeamID,WNum,AvgWpts,TotalWPoints,LNum,AvgLpts,TotalLPoints
0,2003,1102,12.0,15.583333,825.0,16.0,11.250000,778.0
1,2003,1103,13.0,9.384615,1141.0,14.0,7.500000,986.0
2,2003,1104,17.0,13.176471,1270.0,11.0,9.454545,670.0
3,2003,1105,7.0,13.000000,556.0,19.0,11.473684,1310.0
4,2003,1106,13.0,10.384615,888.0,15.0,9.266667,893.0


In [23]:
#winning percentage
df_features_d['WinRatio'] = df_features_d['WNum'] / (df_features_d['WNum'] + df_features_d['LNum'])

#averge ppg
df_features_d['AvgScoreGap'] = (
    (df_features_d['WNum']*df_features_d['AvgWpts']-df_features_d['LNum']*df_features_d['AvgLpts'])
    /(df_features_d['WNum'] + df_features_d['LNum']))


In [24]:
df_features_d.head()

,Season,TeamID,WNum,AvgWpts,TotalWPoints,LNum,AvgLpts,TotalLPoints,WinRatio,AvgScoreGap
0,2003,1102,12.0,15.583333,825.0,16.0,11.250000,778.0,0.428571,0.250000
1,2003,1103,13.0,9.384615,1141.0,14.0,7.500000,986.0,0.481481,0.629630
2,2003,1104,17.0,13.176471,1270.0,11.0,9.454545,670.0,0.607143,4.285714
3,2003,1105,7.0,13.000000,556.0,19.0,11.473684,1310.0,0.269231,-4.884615
4,2003,1106,13.0,10.384615,888.0,15.0,9.266667,893.0,0.464286,-0.142857


In [26]:
df_features_d.columns

Index(['Season', 'TeamID', 'WNum', 'AvgWpts', 'TotalWPoints', 'LNum',
       'AvgLpts', 'TotalLPoints', 'WinRatio', 'AvgScoreGap'],
      dtype='object')

In [27]:
all_df.columns

Index(['Season', 'TeamID', 'Num', 'Score', 'Loc', 'FGM', 'FGA', 'FGM3', 'FGA3',
       'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF', 'TS', 'TO%',
       'FGpct', '3FGpct', 'AST/TO', 'FTpct', 'AST/STL'],
      dtype='object')

In [28]:
cols = ['Season', 'TeamID', 'TS', 'TO', 'AST/TO', 'FTpct', 'AST/STL']
df_self = all_df[cols]

cols=['Season', 'TeamID', 'WinRatio', 'AvgScoreGap']
df_i = df_features_d[cols]

df_final = df_self.merge(df_i)

In [29]:
df_final

,Season,TeamID,TS,TO,AST/TO,FTpct,AST/STL,WinRatio,AvgScoreGap
0,2003,1102,0.605,320,1.138,0.651,2.180,0.428571,0.250000
1,2003,1103,0.586,341,1.205,0.736,2.097,0.481481,0.629630
2,2003,1104,0.522,372,0.911,0.710,1.832,0.607143,4.285714
3,2003,1105,0.504,485,0.779,0.706,1.562,0.269231,-4.884615
4,2003,1106,0.509,477,0.686,0.646,1.397,0.464286,-0.142857
...,...,...,...,...,...,...,...,...,...
6887,2022,1468,0.561,254,1.524,0.750,2.563,0.551724,-2.482759
6888,2022,1469,0.504,405,0.960,0.667,2.358,0.333333,-8.000000
6889,2022,1470,0.514,287,1.021,0.754,1.402,0.392857,-2.678571
6890,2022,1471,0.561,331,1.076,0.747,2.455,0.407407,-3.259259


Tournament Detailed Results

In [30]:
df_tourney_d = pd.read_csv('/Users/seancollins/Desktop/DATA 3000 Project/mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneyDetailedResults.csv')
df_tourney_d.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [31]:
df_tourney_d = df_tourney_d.rename(columns={
   'WTeamID':'W_TeamID', 'LTeamID':'L_TeamID', 'WScore':'W_Score', 'LScore':'L_Score',
    'WLoc':'W_Loc'
})

Feature Engineering

In [32]:
# create team list
cols = ['Season', 'DayNum', 'W_TeamID', 'L_TeamID', 'W_Score', 'L_Score']
df = df_tourney_d[cols].copy()
df

,Season,DayNum,W_TeamID,L_TeamID,W_Score,L_Score
0,2003,134,1421,1411,92,84
1,2003,136,1112,1436,80,51
2,2003,136,1113,1272,84,71
3,2003,136,1141,1166,79,73
4,2003,136,1143,1301,76,74
...,...,...,...,...,...,...
1176,2021,148,1211,1425,85,66
1177,2021,148,1417,1276,51,49
1178,2021,152,1124,1222,78,59
1179,2021,152,1211,1417,93,90


In [33]:
df_seeds = pd.read_csv('/Users/seancollins/Desktop/DATA 3000 Project/mens-march-mania-2022/MDataFiles_Stage2/MNCAATourneySeeds.csv')
df_seeds

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2417,2022,Z13,1151
2418,2022,Z14,1255
2419,2022,Z15,1174
2420,2022,Z16a,1136


In [34]:
# merge with seeds
df = pd.merge(df, df_seeds, how='left', 
    left_on=['Season', 'W_TeamID'], right_on=['Season', 'TeamID']
             ).drop('TeamID', axis=1).rename(columns={'Seed': 'W_Seed'})

df = pd.merge(df, df_seeds, how='left', 
    left_on=['Season', 'L_TeamID'], right_on=['Season', 'TeamID']
             ).drop('TeamID', axis=1).rename(columns={'Seed': 'L_Seed'})


In [35]:
df

,Season,DayNum,W_TeamID,L_TeamID,W_Score,L_Score,W_Seed,L_Seed
0,2003,134,1421,1411,92,84,X16b,X16a
1,2003,136,1112,1436,80,51,Z01,Z16
2,2003,136,1113,1272,84,71,Z10,Z07
3,2003,136,1141,1166,79,73,Z11,Z06
4,2003,136,1143,1301,76,74,W08,W09
...,...,...,...,...,...,...,...,...
1176,2021,148,1211,1425,85,66,X01,X06
1177,2021,148,1417,1276,51,49,W11b,W01
1178,2021,152,1124,1222,78,59,Z01,Y02
1179,2021,152,1211,1417,93,90,X01,W11b


In [36]:
# only get the number, remove the region letter
def treat_seed(seed):
    return int(re.sub("[^0-9]", "", seed))

df['W_Seed'] = df['W_Seed'].apply(treat_seed)
df['L_Seed'] = df['L_Seed'].apply(treat_seed)

In [37]:
df

,Season,DayNum,W_TeamID,L_TeamID,W_Score,L_Score,W_Seed,L_Seed
0,2003,134,1421,1411,92,84,16,16
1,2003,136,1112,1436,80,51,1,16
2,2003,136,1113,1272,84,71,10,7
3,2003,136,1141,1166,79,73,11,6
4,2003,136,1143,1301,76,74,8,9
...,...,...,...,...,...,...,...,...
1176,2021,148,1211,1425,85,66,1,6
1177,2021,148,1417,1276,51,49,11,1
1178,2021,152,1124,1222,78,59,1,2
1179,2021,152,1211,1417,93,90,1,11


In [38]:
# merge with features
df = pd.merge(df, df_final, how='left',
    left_on=['Season', 'W_TeamID'], right_on=['Season', 'TeamID']).drop(columns='TeamID', axis=1)

df = pd.merge(df, df_final, how='left',
    left_on=['Season', 'L_TeamID'], right_on=['Season', 'TeamID']).drop(columns='TeamID', axis=1)

In [39]:
def replace_win_loser(df):
    team_a = df.copy()
    team_b = df.copy()
    
    team_a_dict, team_b_dict = {}, {}
    
    for col in team_a.columns:
        if col.find('W_') == 0:
            new_col_name = str(col).replace('W_', 'A_')
            team_a_dict[col] = new_col_name
        if col.find('L_') == 0:
            new_col_name = str(col).replace('L_', 'B_')    
            team_a_dict[col] = new_col_name
        if col.find('_x') > 0:
            new_col_name = str(col).replace('_x', '_A')
            team_a_dict[col] = new_col_name
        if col.find('_y') > 0:
            new_col_name = str(col).replace('_y', '_B')    
            team_a_dict[col] = new_col_name
            
    for col in team_b.columns:
        if col.find('W_') == 0:
            new_col_name = str(col).replace('W_', 'B_')
            team_b_dict[col] = new_col_name
        if col.find('L_') == 0:
            new_col_name = str(col).replace('L_', 'A_')
            team_b_dict[col] = new_col_name
        if col.find('_x') > 0:
            new_col_name = str(col).replace('_x', '_B')
            team_b_dict[col] = new_col_name
        if col.find('_y') > 0:
            new_col_name = str(col).replace('_y', '_A')    
            team_b_dict[col] = new_col_name

    team_a = team_a.rename(columns = team_a_dict)
    team_b = team_b.rename(columns = team_b_dict)
    
    merged_df = pd.concat([team_a, team_b], axis = 0, sort = False)
    return merged_df

In [40]:
df = replace_win_loser(df).reset_index(drop=True)

In [41]:
# differences between A team and B team

cols_to_diff = ['Seed']

for col in cols_to_diff:
    df[col + 'Diff'] = df['A_'+col] - df['B_'+col]
    
cols_to_diff2 = ['TS', 'TO', 'AST/TO', 'FTpct', 'AST/STL', 'WinRatio','AvgScoreGap']

for col in cols_to_diff2:
    df[col + 'Diff'] = df[col+'_A'] - df[col+'_B']

In [42]:
df.columns

Index(['Season', 'DayNum', 'A_TeamID', 'B_TeamID', 'A_Score', 'B_Score',
       'A_Seed', 'B_Seed', 'TS_A', 'TO_A', 'AST/TO_A', 'FTpct_A', 'AST/STL_A',
       'WinRatio_A', 'AvgScoreGap_A', 'TS_B', 'TO_B', 'AST/TO_B', 'FTpct_B',
       'AST/STL_B', 'WinRatio_B', 'AvgScoreGap_B', 'SeedDiff', 'TSDiff',
       'TODiff', 'AST/TODiff', 'FTpctDiff', 'AST/STLDiff', 'WinRatioDiff',
       'AvgScoreGapDiff'],
      dtype='object')

In [43]:
# Create target variable

df['ScoreDiff'] = df['A_Score'] - df['B_Score']
df['A_Win'] = (df['ScoreDiff'] > 0).astype(int)
df = df.drop(columns=['A_Score', 'B_Score','ScoreDiff'])

In [44]:
df

,Season,DayNum,A_TeamID,B_TeamID,A_Seed,B_Seed,TS_A,TO_A,AST/TO_A,FTpct_A,AST/STL_A,WinRatio_A,AvgScoreGap_A,TS_B,TO_B,AST/TO_B,FTpct_B,AST/STL_B,WinRatio_B,AvgScoreGap_B,SeedDiff,TSDiff,TODiff,AST/TODiff,FTpctDiff,AST/STLDiff,WinRatioDiff,AvgScoreGapDiff,A_Win
0,2003,134,1421,1411,16,16,0.539,470,0.804,0.763,1.844,0.448276,-7.241379,0.538,457,0.932,0.620,2.207,0.600000,1.966667,0,0.001,13,-0.128,0.143,-0.363,-0.151724,-9.208046,1
1,2003,136,1112,1436,1,16,0.555,414,1.193,0.701,2.084,0.892857,14.964286,0.526,408,1.010,0.658,2.070,0.655172,4.655172,-15,0.029,6,0.183,0.043,0.014,0.237685,10.309113,1
2,2003,136,1113,1272,10,7,0.555,406,1.111,0.670,2.987,0.620690,6.793103,0.532,400,1.205,0.654,2.252,0.793103,8.689655,3,0.023,6,-0.094,0.016,0.735,-0.172414,-1.896552,1
3,2003,136,1141,1166,11,6,0.622,529,0.856,0.766,2.199,0.793103,6.103448,0.598,441,1.259,0.693,2.004,0.878788,14.909091,5,0.024,88,-0.403,0.073,0.195,-0.085684,-8.805643,1
4,2003,136,1143,1301,8,9,0.553,411,1.129,0.686,2.442,0.724138,4.724138,0.581,426,1.033,0.770,1.888,0.600000,4.400000,-1,-0.028,-15,0.096,-0.084,0.554,0.124138,0.324138,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2357,2021,148,1425,1211,6,1,0.552,351,1.128,0.647,2.849,0.758621,9.655172,0.640,310,1.535,0.726,2.204,1.000000,23.000000,5,-0.088,41,-0.407,-0.079,0.645,-0.241379,-13.344828,0
2358,2021,148,1276,1417,1,11,0.587,256,1.449,0.778,3.500,0.833333,10.875000,0.558,283,1.251,0.721,2.682,0.653846,4.346154,-10,0.029,-27,0.198,0.057,0.818,0.179487,6.528846,0
2359,2021,152,1222,1124,2,1,0.553,264,1.348,0.743,1.745,0.884615,18.000000,0.598,292,1.401,0.697,1.902,0.916667,17.958333,1,-0.045,-28,-0.053,0.046,-0.157,-0.032051,0.041667,0
2360,2021,152,1417,1211,11,1,0.558,283,1.251,0.721,2.682,0.653846,4.346154,0.640,310,1.535,0.726,2.204,1.000000,23.000000,10,-0.082,-27,-0.284,-0.005,0.478,-0.346154,-18.653846,0


Test Data

In [45]:
df_test = pd.read_csv('/Users/seancollins/Desktop/DATA 3000 Project/mens-march-mania-2022/MDataFiles_Stage2/MSampleSubmissionStage2.csv')
df_test['Season'] = df_test['ID'].apply(lambda x: int(x.split('_')[0]))
df_test['A_TeamID'] = df_test['ID'].apply(lambda x: int(x.split('_')[1]))
df_test['B_TeamID'] = df_test['ID'].apply(lambda x: int(x.split('_')[2]))


In [46]:
# merge with seeds
df_test = pd.merge(df_test, df_seeds, how='left', 
    left_on=['Season', 'A_TeamID'], right_on=['Season', 'TeamID']
             ).drop('TeamID', axis=1).rename(columns={'Seed': 'A_Seed'})

df_test = pd.merge(df_test, df_seeds, how='left', 
    left_on=['Season', 'B_TeamID'], right_on=['Season', 'TeamID']
             ).drop('TeamID', axis=1).rename(columns={'Seed': 'B_Seed'})



In [47]:
df_test

,ID,Pred,Season,A_TeamID,B_TeamID,A_Seed,B_Seed
0,2022_1103_1104,0.5,2022,1103,1104,W13,X06
1,2022_1103_1112,0.5,2022,1103,1112,W13,Z01
2,2022_1103_1116,0.5,2022,1103,1116,W13,X04
3,2022_1103_1120,0.5,2022,1103,1120,W13,Y02
4,2022_1103_1124,0.5,2022,1103,1124,W13,W01
...,...,...,...,...,...,...,...
2273,2022_1458_1461,0.5,2022,1458,1461,Y03,W12b
2274,2022_1458_1463,0.5,2022,1458,1463,Y03,W14
2275,2022_1460_1461,0.5,2022,1460,1461,Z16b,W12b
2276,2022_1460_1463,0.5,2022,1460,1463,Z16b,W14


In [48]:

def treat_seed(seed):
    # print("Original Seed:", seed)
    treated_seed = int(re.sub("[^0-9]", "", str(seed)))
    # print("Treated Seed:", treated_seed)
    return treated_seed

# Apply function and check the results
df_test['A_Seed'] = df_test['A_Seed'].apply(treat_seed)
df_test['B_Seed'] = df_test['B_Seed'].apply(treat_seed)


In [49]:
df_test

,ID,Pred,Season,A_TeamID,B_TeamID,A_Seed,B_Seed
0,2022_1103_1104,0.5,2022,1103,1104,13,6
1,2022_1103_1112,0.5,2022,1103,1112,13,1
2,2022_1103_1116,0.5,2022,1103,1116,13,4
3,2022_1103_1120,0.5,2022,1103,1120,13,2
4,2022_1103_1124,0.5,2022,1103,1124,13,1
...,...,...,...,...,...,...,...
2273,2022_1458_1461,0.5,2022,1458,1461,3,12
2274,2022_1458_1463,0.5,2022,1458,1463,3,14
2275,2022_1460_1461,0.5,2022,1460,1461,16,12
2276,2022_1460_1463,0.5,2022,1460,1463,16,14


In [50]:
# merge with features
df_test = pd.merge(df_test, df_final, how='left',
    left_on=['Season', 'A_TeamID'], right_on=['Season', 'TeamID']).drop(columns='TeamID', axis=1)

df_test = pd.merge(df_test, df_final, how='left',
    left_on=['Season', 'B_TeamID'], right_on=['Season', 'TeamID']).drop(columns='TeamID', axis=1)


In [51]:
cols_to_diff = ['Seed']


for col in cols_to_diff:
    df_test[col + 'Diff'] = df_test['A_'+col] - df_test['B_'+col]
    
cols_to_diff2 = ['TS', 'TO', 'AST/TO', 'FTpct', 'AST/STL', 'WinRatio','AvgScoreGap']

for col in cols_to_diff2:
    df_test[col + 'Diff'] = df_test[col+'_x'] - df_test[col+'_y']


In [52]:
df_test

,ID,Pred,Season,A_TeamID,B_TeamID,A_Seed,B_Seed,TS_x,TO_x,AST/TO_x,FTpct_x,AST/STL_x,WinRatio_x,AvgScoreGap_x,TS_y,TO_y,AST/TO_y,FTpct_y,AST/STL_y,WinRatio_y,AvgScoreGap_y,SeedDiff,TSDiff,TODiff,AST/TODiff,FTpctDiff,AST/STLDiff,WinRatioDiff,AvgScoreGapDiff
0,2022_1103_1104,0.5,2022,1103,1104,13,6,0.567,336,1.062,0.692,2.100,0.709677,5.129032,0.555,451,1.038,0.730,2.089,0.593750,3.562500,7,0.012,-115,0.024,-0.038,0.011,0.115927,1.566532
1,2022_1103_1112,0.5,2022,1103,1112,13,1,0.567,336,1.062,0.692,2.100,0.709677,5.129032,0.597,440,1.539,0.738,2.969,0.911765,17.029412,12,-0.030,-104,-0.477,-0.046,-0.869,-0.202087,-11.900380
2,2022_1103_1116,0.5,2022,1103,1116,13,4,0.567,336,1.062,0.692,2.100,0.709677,5.129032,0.544,401,1.160,0.753,1.824,0.757576,8.393939,9,0.023,-65,-0.098,-0.061,0.276,-0.047898,-3.264907
3,2022_1103_1120,0.5,2022,1103,1120,13,2,0.567,336,1.062,0.692,2.100,0.709677,5.129032,0.544,373,1.247,0.733,1.655,0.843750,11.687500,11,0.023,-37,-0.185,-0.041,0.445,-0.134073,-6.558468
4,2022_1103_1124,0.5,2022,1103,1124,13,1,0.567,336,1.062,0.692,2.100,0.709677,5.129032,0.558,384,1.320,0.698,1.798,0.812500,12.875000,12,0.009,-48,-0.258,-0.006,0.302,-0.102823,-7.745968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2022_1458_1461,0.5,2022,1458,1461,3,12,0.527,261,1.307,0.742,2.079,0.774194,4.161290,0.566,346,1.006,0.725,2.876,0.750000,6.187500,-9,-0.039,-85,0.301,0.017,-0.797,0.024194,-2.026210
2274,2022_1458_1463,0.5,2022,1458,1463,3,14,0.527,261,1.307,0.742,2.079,0.774194,4.161290,0.543,367,0.935,0.737,1.971,0.620690,2.034483,-11,-0.016,-106,0.372,0.005,0.108,0.153504,2.126808
2275,2022_1460_1461,0.5,2022,1460,1461,16,12,0.562,401,1.135,0.766,2.395,0.606061,3.545455,0.566,346,1.006,0.725,2.876,0.750000,6.187500,4,-0.004,55,0.129,0.041,-0.481,-0.143939,-2.642045
2276,2022_1460_1463,0.5,2022,1460,1463,16,14,0.562,401,1.135,0.766,2.395,0.606061,3.545455,0.543,367,0.935,0.737,1.971,0.620690,2.034483,2,0.019,34,0.200,0.029,0.424,-0.014629,1.510972


In [53]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df2 = scaler.fit_transform(df)
df2 = pd.DataFrame(df2, columns=df.columns)

In [54]:
df2[['A_Win','SeedDiff', 'TSDiff', 'TODiff', 'AST/TODiff', 'FTpctDiff', 'AST/STLDiff', 'WinRatioDiff']].corr()


,A_Win,SeedDiff,TSDiff,TODiff,AST/TODiff,FTpctDiff,AST/STLDiff,WinRatioDiff
A_Win,1.000000,-0.485451,0.163807,-0.058256,0.247275,0.041221,-0.002931,0.342834
SeedDiff,-0.485451,1.000000,-0.291486,0.075463,-0.436250,-0.061000,-0.155877,-0.628019
TSDiff,0.163807,-0.291486,1.000000,-0.096011,0.417483,0.461909,0.395118,0.456288
TODiff,-0.058256,0.075463,-0.096011,1.000000,-0.583957,-0.256624,-0.191261,-0.178469
AST/TODiff,0.247275,-0.436250,0.417483,-0.583957,1.000000,0.213358,0.488072,0.442189
FTpctDiff,0.041221,-0.061000,0.461909,-0.256624,0.213358,1.000000,0.202655,0.084762
AST/STLDiff,-0.002931,-0.155877,0.395118,-0.191261,0.488072,0.202655,1.000000,0.106505
WinRatioDiff,0.342834,-0.628019,0.456288,-0.178469,0.442189,0.084762,0.106505,1.000000
